# Qlib Jupyter 入门教程 (美股版)

**版本**: v2.1  
**更新日期**: 2025-12-22  
**适用对象**: Qlib 新手  
**目标**: 通过可视化方式学习 Qlib 的数据处理和模型预测

---

## 📋 版本历史

| 版本 | 日期 | 更新内容 |
|------|------|----------|
| v2.1 | 2025-12-22 | 修复重复初始化报错问题 |
| v2.0 | 2025-12-22 | 新增配置区、优化模型参数、新增评估指标 |
| v1.1 | - | 初始版本 |

---

## 目录
1. ⚙️ 配置区（修改参数从这里开始）
2. 环境准备
3. 初始化 Qlib
4. 数据探索与可视化
5. 因子计算与分析
6. 模型训练与预测
7. 结果可视化与评估

---

## ⚙️ 1. 配置区（所有可调参数集中在这里）

**使用方法**: 修改下面的参数，然后运行整个 Notebook

In [ ]:
#==============================================================================
# 📁 数据配置
#==============================================================================
DATA_PATH = r"F:\qlib_project\data\us_data"  # Windows 路径
# DATA_PATH = "~/.qlib/qlib_data/us_data"    # Mac/Linux 路径

#==============================================================================
# 📅 时间配置
#==============================================================================
# 因子计算时间范围
FACTOR_START = "2019-01-01"
FACTOR_END = "2020-06-30"

# 模型训练/验证/测试时间划分
TRAIN_START = "2018-01-01"
TRAIN_END = "2019-06-30"
VALID_START = "2019-07-01"
VALID_END = "2019-12-31"
TEST_START = "2020-01-01"
TEST_END = "2020-06-30"

#==============================================================================
# 📊 股票配置
#==============================================================================
NUM_STOCKS = 50          # 训练使用的股票数量（越多越慢，建议 50-200）
TOP_K = 5                # Top-K 策略中选择的股票数量

#==============================================================================
# 🤖 模型配置（重要！这些参数影响模型效果）
#==============================================================================
MODEL_CONFIG = {
    "num_boost_round": 500,       # 最大训练轮数
    "early_stopping_rounds": 50,  # 早停轮数（防止过拟合）
    "learning_rate": 0.05,        # 学习率（越小越稳定，但训练更慢）
    "num_leaves": 64,             # 叶子节点数（越大模型越复杂）
    "max_depth": 8,               # 最大深度
    "lambda_l1": 200,             # L1 正则化（防止过拟合）
    "lambda_l2": 200,             # L2 正则化（防止过拟合）
    "colsample_bytree": 0.8,      # 每棵树使用的特征比例
    "subsample": 0.8,             # 每棵树使用的样本比例
}

#==============================================================================
# 🎯 评估配置
#==============================================================================
RISK_FREE_RATE = 0.02    # 无风险利率（用于计算夏普比率）
TRADING_DAYS = 252       # 每年交易日数

print("✅ 配置加载完成！")
print(f"📁 数据路径: {DATA_PATH}")
print(f"📊 股票数量: {NUM_STOCKS}")
print(f"📅 测试期间: {TEST_START} ~ {TEST_END}")
print(f"🤖 模型参数: learning_rate={MODEL_CONFIG['learning_rate']}, num_leaves={MODEL_CONFIG['num_leaves']}")

In [ ]:
# 安装可视化依赖（如果还没安装的话）
# 取消注释下面的行来安装
# !pip install matplotlib seaborn plotly pandas

In [ ]:
# 导入必要的库
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文显示（如果需要）
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置显示选项
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 200)

print("✅ 库导入成功！")

## 2. 初始化 Qlib

运行下面的代码初始化 Qlib（已添加防重复初始化检查）

In [ ]:
import qlib
from qlib.constant import REG_US

# 检查是否已经初始化（防止重复初始化报错）
try:
    from qlib.data import D
    # 尝试一个简单操作，如果成功说明已初始化
    D.calendar(freq="day")[:1]
    print("⚠️ Qlib 已经初始化过了，跳过重复初始化")
except Exception:
    # 未初始化，执行初始化
    qlib.init(
        provider_uri=DATA_PATH,
        region=REG_US,
    )
    print(f"✅ Qlib 初始化成功！")

print(f"📁 数据路径: {DATA_PATH}")

## 3. 数据探索与可视化

### 3.1 查看可用股票列表

In [ ]:
from qlib.data import D

# 获取所有可用股票
all_instruments = D.list_instruments(D.instruments("all"), as_list=True)

print(f"📊 可用股票总数: {len(all_instruments)}")
print(f"\n前20只股票:")
print(all_instruments[:20])

In [ ]:
# 筛选知名股票（示例）
famous_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'JNJ']

# 检查这些股票是否在数据中
available_famous = [s for s in famous_stocks if s in all_instruments]
print(f"知名股票在数据中的: {available_famous}")

### 3.2 获取单只股票的历史数据

In [ ]:
# 选择一只股票进行分析
STOCK = "AAPL"  # 苹果公司

# 如果AAPL不可用，使用第一只股票
if STOCK not in all_instruments:
    STOCK = all_instruments[0]
    print(f"⚠️ AAPL 不可用，使用 {STOCK} 代替")

# 获取股票数据
# $close, $open, $high, $low, $volume 是 Qlib 的标准字段
df = D.features(
    instruments=[STOCK],
    fields=["$close", "$open", "$high", "$low", "$volume"],
    start_time="2019-01-01",
    end_time="2020-12-31",
)

print(f"\n📈 {STOCK} 数据概览:")
print(f"数据形状: {df.shape}")
print(f"时间范围: {df.index.get_level_values('datetime').min()} 到 {df.index.get_level_values('datetime').max()}")
df.head(10)

In [ ]:
# 数据统计
df.describe()

### 3.3 可视化股票价格走势

In [ ]:
# 重置索引以便绘图
df_plot = df.reset_index()

# 创建图表
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 价格走势
ax1 = axes[0]
ax1.plot(df_plot['datetime'], df_plot['$close'], label='Close', color='blue', linewidth=1)
ax1.fill_between(df_plot['datetime'], df_plot['$low'], df_plot['$high'], alpha=0.3, color='gray', label='High-Low Range')
ax1.set_ylabel('Price')
ax1.set_title(f'{STOCK} Stock Price (2019-2020)')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# 成交量
ax2 = axes[1]
ax2.bar(df_plot['datetime'], df_plot['$volume'], color='green', alpha=0.6, width=1)
ax2.set_ylabel('Volume')
ax2.set_xlabel('Date')
ax2.set_title(f'{STOCK} Trading Volume')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.4 多只股票对比

In [ ]:
# 选择多只股票进行对比
stocks_to_compare = all_instruments[:5]  # 取前5只
print(f"对比的股票: {stocks_to_compare}")

# 获取数据
df_multi = D.features(
    instruments=stocks_to_compare,
    fields=["$close"],
    start_time="2019-01-01",
    end_time="2020-12-31",
)

# 数据透视
df_pivot = df_multi.reset_index().pivot(index='datetime', columns='instrument', values='$close')

# 归一化（以第一天为基准）
df_normalized = df_pivot / df_pivot.iloc[0] * 100

print("\n归一化后的数据（第一天 = 100）:")
df_normalized.head()

In [ ]:
# 绘制多股票对比图
plt.figure(figsize=(14, 6))

for col in df_normalized.columns:
    plt.plot(df_normalized.index, df_normalized[col], label=col, linewidth=1.5)

plt.axhline(y=100, color='black', linestyle='--', alpha=0.5, label='Baseline (100)')
plt.xlabel('Date')
plt.ylabel('Normalized Price (Base=100)')
plt.title('Stock Price Comparison (Normalized)')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. 因子计算与分析

### 4.1 使用 Qlib 表达式计算因子

In [ ]:
# Qlib 支持使用表达式语法计算因子
# 常用表达式:
# - $close: 收盘价
# - Ref($close, 1): 前一天收盘价
# - Mean($close, 5): 5日均值
# - Std($close, 5): 5日标准差
# - Return($close, 1): 1日收益率

# 计算一些常用因子
factors = D.features(
    instruments=[STOCK],
    fields=[
        "$close",                           # 收盘价
        "Ref($close, 1)",                   # 前一日收盘价
        "$close / Ref($close, 1) - 1",      # 日收益率
        "Mean($close, 5)",                  # 5日均线
        "Mean($close, 20)",                 # 20日均线
        "Std($close, 20)",                  # 20日波动率
        "($close - Mean($close, 20)) / Std($close, 20)",  # 布林带位置
    ],
    start_time="2019-01-01",
    end_time="2020-12-31",
)

# 重命名列
factors.columns = ['close', 'prev_close', 'return', 'ma5', 'ma20', 'std20', 'bollinger']

print("因子数据:")
factors.tail(10)

In [ ]:
# 可视化均线
df_factors = factors.reset_index()

plt.figure(figsize=(14, 6))
plt.plot(df_factors['datetime'], df_factors['close'], label='Close Price', linewidth=1)
plt.plot(df_factors['datetime'], df_factors['ma5'], label='MA5', linewidth=1, linestyle='--')
plt.plot(df_factors['datetime'], df_factors['ma20'], label='MA20', linewidth=1.5, linestyle='-')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title(f'{STOCK} Price with Moving Averages')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 收益率分布
returns = factors['return'].dropna()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 直方图
ax1 = axes[0]
ax1.hist(returns, bins=50, edgecolor='black', alpha=0.7)
ax1.axvline(x=0, color='red', linestyle='--', label='Zero')
ax1.axvline(x=returns.mean(), color='green', linestyle='-', label=f'Mean: {returns.mean():.4f}')
ax1.set_xlabel('Daily Return')
ax1.set_ylabel('Frequency')
ax1.set_title(f'{STOCK} Daily Return Distribution')
ax1.legend()

# 累计收益
ax2 = axes[1]
cumulative_return = (1 + returns).cumprod()
ax2.plot(cumulative_return.index.get_level_values('datetime'), cumulative_return.values)
ax2.set_xlabel('Date')
ax2.set_ylabel('Cumulative Return')
ax2.set_title(f'{STOCK} Cumulative Return')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 收益率统计:")
print(f"平均日收益: {returns.mean():.4%}")
print(f"收益标准差: {returns.std():.4%}")
print(f"最大日收益: {returns.max():.4%}")
print(f"最小日收益: {returns.min():.4%}")
print(f"夏普比率 (年化): {returns.mean() / returns.std() * np.sqrt(252):.2f}")

### 4.2 使用 Alpha158 因子处理器

In [ ]:
from qlib.contrib.data.handler import Alpha158

# 选择少量股票以加快计算
selected_stocks = all_instruments[:20]  # 只用20只股票

print(f"📊 正在为 {len(selected_stocks)} 只股票计算 Alpha158 因子...")
print("(这可能需要几分钟)")

# 创建因子处理器
handler = Alpha158(
    instruments=selected_stocks,
    start_time="2019-01-01",
    end_time="2020-06-30",
    fit_start_time="2019-01-01",
    fit_end_time="2019-12-31",
)

print("✅ 因子计算完成！")

In [ ]:
# 获取因子数据
alpha_data = handler.fetch()

print(f"因子数据形状: {alpha_data.shape}")
print(f"\n因子列表 (共 {len(alpha_data.columns)} 个):")
print(list(alpha_data.columns[:20]))  # 显示前20个因子
print("...")

In [ ]:
# 查看因子数据
alpha_data.head(10)

In [ ]:
# 因子相关性热力图
# 选择部分因子进行分析
sample_factors = alpha_data.iloc[:, :15]  # 前15个因子

# 计算相关性矩阵
corr_matrix = sample_factors.corr()

# 绘制热力图
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            square=True, linewidths=0.5)
plt.title('Factor Correlation Heatmap')
plt.tight_layout()
plt.show()

from qlib.data.dataset import DatasetH
from qlib.contrib.data.handler import Alpha158

# 使用配置区的股票数量
train_stocks = all_instruments[:NUM_STOCKS]

print(f"📊 正在准备数据集...")
print(f"股票数量: {len(train_stocks)}")
print(f"训练期: {TRAIN_START} ~ {TRAIN_END}")
print(f"验证期: {VALID_START} ~ {VALID_END}")
print(f"测试期: {TEST_START} ~ {TEST_END}")

# 创建数据集（使用配置区的时间参数）
handler = Alpha158(
    instruments=train_stocks,
    start_time=TRAIN_START,
    end_time=TEST_END,
    fit_start_time=TRAIN_START,
    fit_end_time=TRAIN_END,
)

dataset = DatasetH(
    handler=handler,
    segments={
        "train": (TRAIN_START, TRAIN_END),
        "valid": (VALID_START, VALID_END),
        "test": (TEST_START, TEST_END),
    }
)

print("✅ 数据集准备完成！")

In [ ]:
from qlib.data.dataset import DatasetH
from qlib.contrib.data.handler import Alpha158

# 使用较少的股票以加快速度
train_stocks = all_instruments[:50]  # 50只股票

print(f"📊 正在准备数据集...")
print(f"股票数量: {len(train_stocks)}")

# 创建数据集
handler = Alpha158(
    instruments=train_stocks,
    start_time="2018-01-01",
    end_time="2020-06-30",
    fit_start_time="2018-01-01",
    fit_end_time="2019-06-30",
)

dataset = DatasetH(
    handler=handler,
    segments={
        "train": ("2018-01-01", "2019-06-30"),  # 训练集: 1.5年
        "valid": ("2019-07-01", "2019-12-31"),  # 验证集: 半年
        "test": ("2020-01-01", "2020-06-30"),   # 测试集: 半年
    }
)

print("✅ 数据集准备完成！")

In [ ]:
# 查看数据集信息
train_data = dataset.prepare("train", col_set=["feature", "label"])
valid_data = dataset.prepare("valid", col_set=["feature", "label"])
test_data = dataset.prepare("test", col_set=["feature", "label"])

print(f"训练集: {train_data.shape}")
print(f"验证集: {valid_data.shape}")
print(f"测试集: {test_data.shape}")

from qlib.contrib.model.gbdt import LGBModel

print("🚀 开始训练模型...")
print(f"📋 模型参数: {MODEL_CONFIG}")

# 创建并训练模型（使用配置区的参数）
model = LGBModel(
    loss="mse",
    num_boost_round=MODEL_CONFIG["num_boost_round"],
    early_stopping_rounds=MODEL_CONFIG["early_stopping_rounds"],
    num_leaves=MODEL_CONFIG["num_leaves"],
    learning_rate=MODEL_CONFIG["learning_rate"],
    max_depth=MODEL_CONFIG.get("max_depth", 8),
    lambda_l1=MODEL_CONFIG.get("lambda_l1", 200),
    lambda_l2=MODEL_CONFIG.get("lambda_l2", 200),
    colsample_bytree=MODEL_CONFIG.get("colsample_bytree", 0.8),
    subsample=MODEL_CONFIG.get("subsample", 0.8),
)

model.fit(dataset)

print("✅ 模型训练完成！")

In [ ]:
from qlib.contrib.model.gbdt import LGBModel

print("🚀 开始训练模型...")

# 创建并训练模型
model = LGBModel(
    loss="mse",
    num_boost_round=100,
    early_stopping_rounds=20,
    num_leaves=64,
    learning_rate=0.1,
)

model.fit(dataset)

print("✅ 模型训练完成！")

### 5.3 生成预测

In [ ]:
# 在测试集上进行预测
predictions = model.predict(dataset, segment="test")

print(f"预测结果数量: {len(predictions)}")
print(f"\n预测结果示例:")
print(predictions.head(20))

In [ ]:
# 获取真实标签
test_labels = test_data["label"].squeeze()

# 确保索引对齐
common_idx = predictions.index.intersection(test_labels.index)
pred_aligned = predictions.loc[common_idx]
label_aligned = test_labels.loc[common_idx]

print(f"对齐后的数据量: {len(common_idx)}")

## 6. 结果可视化

### 6.1 预测值 vs 真实值

In [ ]:
# 散点图: 预测值 vs 真实值
plt.figure(figsize=(10, 8))

# 随机采样以避免过多点
sample_size = min(5000, len(pred_aligned))
sample_idx = np.random.choice(len(pred_aligned), sample_size, replace=False)

plt.scatter(
    label_aligned.iloc[sample_idx], 
    pred_aligned.iloc[sample_idx],
    alpha=0.3,
    s=10
)

# 添加对角线
min_val = min(label_aligned.min(), pred_aligned.min())
max_val = max(label_aligned.max(), pred_aligned.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction')

plt.xlabel('Actual Return')
plt.ylabel('Predicted Return')
plt.title('Predicted vs Actual Returns')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 计算相关系数
correlation = pred_aligned.corr(label_aligned)
print(f"\n📊 预测与实际的相关系数: {correlation:.4f}")

### 6.2 按日期分析预测质量

In [ ]:
# 创建对比 DataFrame
comparison = pd.DataFrame({
    'prediction': pred_aligned,
    'actual': label_aligned
})

# 按日期计算日均相关系数
daily_corr = comparison.groupby(level='datetime').apply(
    lambda x: x['prediction'].corr(x['actual'])
).dropna()

print(f"日均相关系数统计:")
print(daily_corr.describe())

In [ ]:
# 可视化日相关系数
plt.figure(figsize=(14, 5))

plt.plot(daily_corr.index, daily_corr.values, linewidth=1, alpha=0.7)
plt.axhline(y=daily_corr.mean(), color='red', linestyle='--', label=f'Mean: {daily_corr.mean():.4f}')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)

plt.xlabel('Date')
plt.ylabel('Daily IC (Correlation)')
plt.title('Daily Information Coefficient (IC)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 6.3 分组回测分析

In [ ]:
# 每天按预测值分成5组（处理重复值）
def assign_group(x):
    try:
        # duplicates='drop' 处理重复的分位数边界
        return pd.qcut(x['prediction'], q=5, labels=['G1_Low', 'G2', 'G3', 'G4', 'G5_High'], duplicates='drop')
    except ValueError:
        # 如果分组失败（数据太少或太集中），返回 NaN
        return pd.Series([np.nan] * len(x), index=x.index)

comparison['group'] = comparison.groupby(level='datetime').apply(assign_group).droplevel(0)

# 删除无法分组的数据
comparison_valid = comparison.dropna(subset=['group'])

# 计算每组的平均实际收益
group_returns = comparison_valid.groupby(['datetime', 'group'])['actual'].mean().unstack()

print("各组平均日收益:")
print(group_returns.mean())

In [ ]:
# 可视化各组累计收益
cumulative_returns = (1 + group_returns).cumprod()

plt.figure(figsize=(14, 6))

# 动态分配颜色，避免列数不匹配
color_map = {
    'G1_Low': 'red',
    'G2': 'orange', 
    'G3': 'gray',
    'G4': 'lightgreen',
    'G5_High': 'green'
}

for col in cumulative_returns.columns:
    color = color_map.get(col, 'blue')  # 默认蓝色
    plt.plot(cumulative_returns.index, cumulative_returns[col], 
             label=col, linewidth=2, color=color)

plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.title('Cumulative Returns by Prediction Quintile')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 分析结论:")
print("如果 G5_High (预测最高组) 的累计收益明显高于 G1_Low (预测最低组),")
print("说明模型具有预测能力。")

# 每天选择预测最高的 K 只股票（使用配置区的 TOP_K）
K = TOP_K

def get_topk_return(group):
    top_k = group.nlargest(K, 'prediction')
    return top_k['actual'].mean()

# 使用清洗后的数据计算策略收益
strategy_returns = comparison_valid.groupby(level='datetime').apply(get_topk_return)

# 计算基准收益（所有股票等权）
benchmark_returns = comparison_valid.groupby(level='datetime')['actual'].mean()

print(f"Top-{K} 策略日均收益: {strategy_returns.mean():.4%}")
print(f"基准(等权)日均收益: {benchmark_returns.mean():.4%}")
print(f"超额收益: {(strategy_returns.mean() - benchmark_returns.mean()):.4%}")

In [ ]:
# 可视化策略 vs 基准
cum_strategy = (1 + strategy_returns).cumprod()
cum_benchmark = (1 + benchmark_returns).cumprod()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. 累计收益对比
ax1 = axes[0, 0]
ax1.plot(cum_strategy.index, cum_strategy.values, label=f'Top-{K} Strategy', linewidth=2, color='blue')
ax1.plot(cum_benchmark.index, cum_benchmark.values, label='Benchmark', linewidth=2, color='gray', linestyle='--')
ax1.set_title('Cumulative Returns')
ax1.set_ylabel('Cumulative Return')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. 回撤曲线
ax2 = axes[0, 1]
peak = cum_strategy.expanding().max()
drawdown = (cum_strategy - peak) / peak
ax2.fill_between(drawdown.index, 0, drawdown.values, color='red', alpha=0.5)
ax2.set_title('Strategy Drawdown')
ax2.set_ylabel('Drawdown')
ax2.grid(True, alpha=0.3)

# 3. 日收益分布
ax3 = axes[1, 0]
ax3.hist(strategy_returns, bins=30, edgecolor='black', alpha=0.7, color='blue', label='Strategy')
ax3.hist(benchmark_returns, bins=30, edgecolor='black', alpha=0.5, color='gray', label='Benchmark')
ax3.axvline(x=0, color='red', linestyle='--')
ax3.set_title('Daily Return Distribution')
ax3.set_xlabel('Daily Return')
ax3.legend()

# 4. 滚动夏普比率
ax4 = axes[1, 1]
rolling_sharpe = strategy_returns.rolling(20).apply(
    lambda x: x.mean() / x.std() * np.sqrt(TRADING_DAYS) if x.std() > 0 else 0
)
ax4.plot(rolling_sharpe.index, rolling_sharpe.values, linewidth=1.5, color='blue')
ax4.axhline(y=1, color='green', linestyle='--', alpha=0.5, label='Sharpe=1')
ax4.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax4.set_title('Rolling Sharpe Ratio (20-day)')
ax4.set_ylabel('Sharpe Ratio')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#==============================================================================
# 📊 综合评估指标
#==============================================================================
# 收益指标
total_return = cum_strategy.iloc[-1] - 1
annual_return = strategy_returns.mean() * TRADING_DAYS
annual_vol = strategy_returns.std() * np.sqrt(TRADING_DAYS)
sharpe = (annual_return - RISK_FREE_RATE) / annual_vol

# 风险指标
max_drawdown = drawdown.min()
win_rate = (strategy_returns > 0).sum() / len(strategy_returns)
profit_loss_ratio = abs(strategy_returns[strategy_returns > 0].mean() / strategy_returns[strategy_returns < 0].mean())

# IC 指标
ic_mean = daily_corr.mean()
ic_std = daily_corr.std()
ir = ic_mean / ic_std if ic_std > 0 else 0

# 超额收益
excess_return = (strategy_returns - benchmark_returns).mean() * TRADING_DAYS

print("=" * 60)
print("📊 策略综合评估报告")
print("=" * 60)

print("\n【收益指标】")
print(f"  总收益率: {total_return:.2%}")
print(f"  年化收益率: {annual_return:.2%}")
print(f"  超额年化收益: {excess_return:.2%}")

print("\n【风险指标】")
print(f"  年化波动率: {annual_vol:.2%}")
print(f"  最大回撤: {max_drawdown:.2%}")
print(f"  夏普比率: {sharpe:.2f}")

print("\n【交易指标】")
print(f"  胜率: {win_rate:.2%}")
print(f"  盈亏比: {profit_loss_ratio:.2f}")

print("\n【预测能力】")
print(f"  IC 均值: {ic_mean:.4f}")
print(f"  IC 标准差: {ic_std:.4f}")
print(f"  IR (IC均值/IC标准差): {ir:.4f}")

print("\n" + "=" * 60)
# 综合评级
if sharpe >= 2.0 and ic_mean >= 0.03:
    grade = "A (优秀)"
elif sharpe >= 1.5 and ic_mean >= 0.02:
    grade = "B+ (良好)"
elif sharpe >= 1.0:
    grade = "B (中等)"
elif sharpe >= 0.5:
    grade = "C (及格)"
else:
    grade = "D (需改进)"
print(f"📈 综合评级: {grade}")
print("=" * 60)

In [ ]:
## 8. 总结与下一步

恭喜你完成了 Qlib Jupyter 入门教程 v2.0！

### ✅ 你已经学会了:

| 模块 | 技能 |
|------|------|
| 配置管理 | 通过配置区一键调整所有参数 |
| 数据探索 | 加载和可视化股票数据 |
| 因子计算 | 使用 Qlib 表达式和 Alpha158 |
| 模型训练 | 训练 LightGBM 并防止过拟合 |
| 策略回测 | Top-K 策略和基准对比 |
| 评估指标 | 夏普比率、IC、IR、最大回撤等 |

### 🔧 参数调优建议:

| 参数 | 当前值 | 调整方向 |
|------|--------|----------|
| `NUM_STOCKS` | 50 | ↑ 增加可提升稳定性 |
| `TOP_K` | 5 | ↑ 增加可降低波动 |
| `learning_rate` | 0.05 | ↓ 降低可减少过拟合 |
| `lambda_l1/l2` | 200 | ↑ 增加可加强正则化 |

### 📚 进阶学习:

1. **Qlib_进阶教程_可视化_因子_回测.md** - 深入学习因子和回测
2. **尝试其他模型**: XGBoost, CatBoost, 神经网络
3. **自定义因子**: 编写自己的 Alpha 因子
4. **完整回测**: 使用 Qlib 的 Backtest 模块

### 🔗 参考资源:
- [Qlib 官方文档](https://qlib.readthedocs.io/)
- [Qlib GitHub](https://github.com/microsoft/qlib)
- [LightGBM 参数说明](https://lightgbm.readthedocs.io/en/latest/Parameters.html)

## 7. 下一步

恭喜你完成了 Qlib Jupyter 入门教程！你现在已经学会了:

1. ✅ 初始化 Qlib 并加载数据
2. ✅ 探索和可视化股票数据
3. ✅ 使用 Qlib 表达式计算因子
4. ✅ 使用 Alpha158 因子处理器
5. ✅ 训练 LightGBM 模型
6. ✅ 可视化预测结果
7. ✅ 进行简单的策略回测

### 进阶学习方向:

- **更多模型**: 尝试其他模型如 `XGBModel`, `DNNModel`, `TabNetModel`
- **自定义因子**: 学习如何编写自己的因子表达式
- **完整回测**: 使用 Qlib 的 `Backtest` 模块进行完整的策略回测
- **MLflow 集成**: 使用 Qlib 的实验管理功能

### 参考资源:
- [Qlib 官方文档](https://qlib.readthedocs.io/)
- [Qlib GitHub](https://github.com/microsoft/qlib)